In [1]:
import os
%pwd

'c:\\Users\\ravi2\\projects\\mlops_project\\mlops_project_with_deployment\\research'

In [2]:
# os.chdir("../")
%pwd

'c:\\Users\\ravi2\\projects\\mlops_project\\mlops_project_with_deployment'

In [3]:
import os

os.environ["MLFLOW_TRACKING_URI"] ="https://dagshub.com/01ravi-kumar/mlops_project_with_deployment.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="01ravi-kumar"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6af292beeff34b2f2f708188de42e80d70198d91"


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str
    mlflow_uri: str

In [9]:
from src.MLOps_project_dev_v1.constants import *
from src.MLOps_project_dev_v1.utils.common import read_yaml,create_directories,save_json 

In [14]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
            ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self):
        config = self.config.model_evaluation
        param = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir, 
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            all_params= param,
            metric_file_name=config.metric_file_name,
            target_column= schema.name,
            mlflow_uri="https://dagshub.com/01ravi-kumar/mlops_project_with_deployment.mlflow",
        )

        return model_evaluation_config


In [7]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from urllib.parse import urlparse
import joblib
import mlflow
import mlflow.sklearn

In [16]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)

        return rmse,mae,r2


    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
    


In [18]:
try:
    config= ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e


c:\Users\ravi2\projects\mlops_project\mlops_project_with_deployment
[2023-09-17 12:45:32,881: INFO: common: yaml file:config\config.yaml loaded sucessfully]
c:\Users\ravi2\projects\mlops_project\mlops_project_with_deployment
[2023-09-17 12:45:32,883: INFO: common: yaml file:params.yaml loaded sucessfully]
c:\Users\ravi2\projects\mlops_project\mlops_project_with_deployment
[2023-09-17 12:45:32,883: INFO: common: yaml file:schema.yaml loaded sucessfully]
[2023-09-17 12:45:32,883: INFO: common: created directory at: artifacts]
[2023-09-17 12:45:32,890: INFO: common: created directory at: artifacts/model_evaluation]
[2023-09-17 12:45:33,375: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2023/09/17 12:45:52 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.
